# Homework 4: machine learning

## Part 1: data processing

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [ ]:
dyads = pd.read_csv("CrowdstormingDataJuly1st.csv", index_col=0)
dyads = dyads[dyads['rater1'].isnull() == False]
print(dyads.shape)
dyads.head()

In [ ]:
group_players = dyads.groupby(['player'])

players = group_players.agg({'games':'sum', 'victories':'sum', #'club':'first','leagueCountry':'first',
                       'defeats':'sum', 'ties': 'sum', 'goals':'sum', 'redCards':'sum', 'yellowCards':'sum'})
print(players.shape)
players.head()
# normalize data? example victories as a %

In [ ]:
le = preprocessing.LabelEncoder()
# transform categorical data into numerical values
categorical_values = ['club', 'leagueCountry', 'position']
for name in categorical_values:
    categorie = group_players.agg({name:'first'})
    le.fit(categorie.as_matrix().flatten().tolist())
    players[name] = le.transform(categorie.as_matrix().flatten().tolist())

players.head()

In [ ]:
skin_color = dyads.groupby(['player'])
skin_color = skin_color.agg({'rater1' : 'first'})
skin_color.head()

In [ ]:
X = players.as_matrix().tolist()
Y = skin_color.as_matrix().flatten().tolist()
# map 0.25 to 1 etc
Y = list(map((lambda x: x*4), Y))

In [ ]:
X_train = X[:1000]
X_test = X[1000:]
Y_train = Y[:1000]
Y_test = Y[1000:]

In [ ]:
clf = RandomForestClassifier(n_estimators=200)
clf = clf.fit(X_train, Y_train)

In [ ]:
Y_predict = clf.predict(X_test)
print("accurancy predictions: ",(Y_test - Y_predict).tolist().count(0) / len(Y_predict))
#Y_test - Y_predict